In [ ]:
import pandas as pd
import numpy as np
from PIL import Image

from keras.layers import Input, Conv2D, Dense, LeakyReLU, Dropout, Flatten, PReLU
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam

import os

Завантаження даних

In [ ]:
# load data 'gender-prediction.zip'
if not os.path.exists('gender-prediction.zip'):
    from google.colab import files
    files.upload()

In [ ]:
!unzip --qq -n gender-prediction.zip

Підготовка даних

In [ ]:
def load_data(img_dir, labels_csv, scale=True):
    labels = pd.read_csv(labels_csv)
    X = np.array([np.array(Image.open(os.path.join(img_dir, filename)))
                 for filename in labels['Filename']])
    y = np.array(labels['Male'])

    if scale:
        X = X / 255 - 0.5
        
    return X, y


DATA_DIR = 'gender-prediction'
IMG_DIR = os.path.join(DATA_DIR, 'img') 

X_train, y_train = load_data(IMG_DIR, os.path.join(DATA_DIR, 'train.csv'))
X_val, y_val = load_data(IMG_DIR, os.path.join(DATA_DIR, 'val.csv'))
X_test, y_test = load_data(IMG_DIR, os.path.join(DATA_DIR, 'test.csv'))

In [ ]:
X_train.shape, X_val.shape, X_test.shape

((1200, 350, 350, 3), (150, 350, 350, 3), (150, 350, 350, 3))

Побудова моделі

In [ ]:
input_shape = X_train.shape[1:]

model = Sequential([
    Input(shape=input_shape),
    Conv2D(3, 3, padding='same'),
    PReLU(),
    Conv2D(8, 3, padding='same'),
    PReLU(),
    Conv2D(16, 3, strides=2, padding='same'),
    PReLU(),

    Conv2D(32, 3, padding='same'),
    PReLU(),
    Conv2D(32, 3, strides=2, padding='same'),
    PReLU(),

    Conv2D(64, 5, padding='same'),
    PReLU(),
    Conv2D(128, 5, strides=2, padding='same'),
    PReLU(),
    Dropout(0.3),

    Flatten(),
    Dense(256),
    PReLU(),
    Dense(64),
    PReLU(),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')
_ = model.fit(X_train, y_train, batch_size=18, epochs=15, validation_data=(X_val, y_val), shuffle=True)

Epoch 1/15
67/67 [==============================] - 23s 318ms/step - loss: 0.6355 - accuracy: 0.6692 - val_loss: 0.5514 - val_accuracy: 0.7600
Epoch 2/15
67/67 [==============================] - 20s 306ms/step - loss: 0.4779 - accuracy: 0.7758 - val_loss: 0.5362 - val_accuracy: 0.7267
Epoch 3/15
67/67 [==============================] - 21s 314ms/step - loss: 0.3961 - accuracy: 0.8142 - val_loss: 0.4294 - val_accuracy: 0.8000
Epoch 4/15
67/67 [==============================] - 21s 306ms/step - loss: 0.3515 - accuracy: 0.8533 - val_loss: 0.4756 - val_accuracy: 0.8133
Epoch 5/15
67/67 [==============================] - 21s 306ms/step - loss: 0.3478 - accuracy: 0.8542 - val_loss: 0.3663 - val_accuracy: 0.8533
Epoch 6/15
67/67 [==============================] - 21s 306ms/step - loss: 0.2844 - accuracy: 0.8775 - val_loss: 0.2954 - val_accuracy: 0.8800
Epoch 7/15
67/67 [==============================] - 21s 306ms/step - loss: 0.2405 - accuracy: 0.9050 - val_loss: 0.2576 - val_accuracy: 0.8800

In [ ]:
_ = model.evaluate(X_test, y_test)

5/5 [==============================] - 1s 155ms/step - loss: 0.7553 - accuracy: 0.8933
